In [20]:
 #preliminary 
from tkinter import *
from tkinter import ttk
from tkinter.messagebox import showinfo
import mysql.connector

connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="dtsc221",  
    database="world_health_db"
)
cursor = connection.cursor()

#country list 
def get_country_names():
    query = "SELECT DISTINCT country FROM country_facts"  
    cursor.execute(query)
    countries = [row[0] for row in cursor.fetchall()]
    return countries

#selecting health area and then querying
def query_health_area():
    selected_item = health_area_tree.selection()[0]
    health_area = health_area_tree.item(selected_item, "text")
    print("Querying data for:", health_area)
    execute_health_query(health_area)

#querying 
def execute_health_query(health_area):
    if health_area == "Probability of death in children under 5":
        query = "SELECT country, year, prob_death_under5 FROM child_health ORDER BY country , year"
    elif health_area == "Death in children (per 100000)":
        query = "SELECT country, year, deaths_per100000 FROM child_health ORDER BY country , year"
    elif health_area == "Underweight Children (% of children under 5)":
        query = "SELECT country, year, pct_underweight_under5 FROM child_health ORDER BY country , year"
    elif health_area == "Maternal deaths":
        query = "SELECT country, year, maternal_deaths FROM maternal_health ORDER BY country , year"
    elif health_area == "Lifetime risk of maternal death (%)":
        query = "SELECT country, year, Lifetime_risk_of_maternal_death_prct FROM maternal_health ORDER BY country, year "
    elif health_area == "Skilled nurse at delivery (%)":
        query = "SELECT country, year, Nurse_at_birth_prct FROM maternal_health ORDER BY country , year "
    elif health_area == "Proportion of women with access to modern contraceptives":
        query = "SELECT country, year, prop_women_access_contraceptive FROM maternal_health ORDER BY country , year"
    elif health_area == "Death by Air Pollution":
        query = "SELECT country, year, air_pollution FROM death_factors ORDER BY country , year " 
    elif health_area == "Death by Alcoholism":
        query = "SELECT country, year, alcohol FROM death_factors ORDER BY country , year" 
    elif health_area == "Death by Obesity":
        query = "SELECT country, year, obesity FROM death_factors ORDER BY country , year" 
    elif health_area == "Death by Smoking":
        query = "SELECT country, year, Smoking FROM death_factors ORDER BY country, year " 
    elif health_area == "Death by Unsafe Water":
        query = "SELECT country, year, unsafe_water FROM death_factors ORDER BY country , year " 
    elif health_area == "Life expectancy at birth":
        query = "SELECT country, year, le_at_birth FROM life_expectancy ORDER BY country, year" 
    elif health_area == "Female life expectancy":
        query = "SELECT country, year, female_le FROM life_expectancy ORDER BY country , year " 
    elif health_area == "Male life expectancy":
        query = "SELECT country, year, male_le FROM life_expectancy ORDER BY country, year" 
    elif health_area == "Healthy Life expectancy":
        query = "SELECT country, year, health_le FROM life_expectancy ORDER BY country, year  " 
    elif health_area == "Hib3":
        query = "SELECT country, year, hib3 FROM vaccination ORDER BY country, year LIMIT 10" 
    elif health_area == "HepB3":
        query = "SELECT country, year, hepb3 FROM vaccination ORDER BY country, year " 
    elif health_area == "RotaC":
        query = "SELECT country, year, rotac FROM vaccination ORDER BY country, year " 
    elif health_area == "RCV1":
        query = "SELECT country, year, rcv1 FROM vaccination ORDER BY country, year" 
    else:
        showinfo("Query Error", "Selected health area is not yet supported.")
        return
    
    cursor.execute(query)
    data = cursor.fetchall()

    #results health_area_results
    result_window = Toplevel()
    result_window.title("Query Result")
    result_window.geometry("600x400")
    
    #tree view
    tree = ttk.Treeview(result_window, show="headings")
    tree.pack(fill="both", expand=True) 
    tree["columns"] = ("Country", "Year", "Value")
    tree.heading("Country", text="Country", anchor="center")
    tree.heading("Year", text="Year", anchor="center")
    tree.heading("Value", text=health_area, anchor="center")

    #insert in tree
    for row in data:
        tree.insert("", "end", values=row)

    #FILTER BY COUNTRY
    def filter_by_country():
        selected_country = country_combobox.get()
        if selected_country:
            # Filter data based on the selected country
            filtered_data = [row for row in data if row[0] == selected_country]
            for item in tree.get_children():
                tree.delete(item)
            # Insert filtered data into treeview
            for row in filtered_data:
                tree.insert("", "end", values=row)
    
    #Country combobox
    country_var = StringVar()
    country_combobox = ttk.Combobox(result_window, textvariable=country_var)
    country_combobox['values'] = sorted(set(row[0] for row in data))
    country_combobox.pack(side=TOP, padx=5, pady=10)  # Change the side to TOP
    
    #Filter by Country button
    filter_country_button = Button(result_window, text="Filter by Country", command=filter_by_country)
    filter_country_button.pack(side=TOP, padx=5)  # Change the side to TOP


def execute_query(selected_country):
   #QUERY for country selection
    query = f"""
    SELECT country_facts.country, death_factors.year, child_health.prob_death_under5, death_factors.alcohol, death_factors.air_pollution
    FROM country_facts
    JOIN child_health ON country_facts.code=child_health.code
    JOIN death_factors ON child_health.code=death_factors.code
    WHERE death_factors.country='{selected_country}' ORDER BY death_factors.year
    """
    cursor.execute(query)
    data = cursor.fetchall()

    #Result window:
    result_window = Toplevel()
    result_window.title("Query Result")
    result_window.geometry("800x400")

    #tree view
    tree = ttk.Treeview(result_window, show="headings")
    tree.pack(padx=10, pady=10)

    tree["columns"] = ("Country", "Year", "Probability of death in children under 5", "Death by Alcoholism", "Death by Air Pollution")
    tree.heading("Country", text="Country", anchor="center")
    tree.heading("Year", text="Year", anchor="center")
    tree.heading("Probability of death in children under 5", text="Probability of death in children under 5", anchor="center")
    tree.heading("Death by Alcoholism", text="Death by Alcoholism", anchor="center")
    tree.heading("Death by Air Pollution", text="Death by Air Pollution", anchor="center")

    #insert in tree
    for row in data:
        tree.insert("", "end", values=row)

#Explore button on homepage
def explore_db():
    selection = db_combobox.get()

#by country:
    if selection == "By Country":
        country_window = Toplevel(root)
        country_window.title("Select Country")
        country_window.geometry("300x400") 
        
        countries = get_country_names()
        countries.sort()
        
        selected_country = StringVar()
        selected_country.set("")  #empty combobox in the start
        
        country_listbox = Listbox(country_window, selectmode=SINGLE)
        for country in countries:
            country_listbox.insert(END, country)
        country_listbox.pack(fill=BOTH, expand=True)  
        
        ok_button = Button(country_window, text="Explore", command=lambda: execute_query(country_listbox.get(ACTIVE)))
        ok_button.pack(pady=5)
        
# by health area:
    elif selection == "By Health Area":
        health_area_window = Toplevel(root)
        health_area_window.title("Select Health Area")
        
        #tree for health area:
        global health_area_tree
        health_area_tree = ttk.Treeview(health_area_window)
        health_area_tree.pack(padx=10, pady=10)
        
        health_areas = {
            "Child Health": ["Probability of death in children under 5", "Death in children (per 100000)", "Underweight Children (% of children under 5)"],
            "Maternal Health": ["Maternal deaths", "Lifetime risk of maternal death (%)", "Skilled nurse at delivery (%)", "Proportion of women with access to modern contraceptives"],
            "Death Factors": ["Death by Air Pollution", "Death by Alcoholism", "Death by Obesity", "Death by Smoking", "Death by Unsafe Water"],
            "Life Expectancy": ["Life expectancy at birth", "Healthy Life expectancy", "Female life expectancy", "Male life expectancy"],
            "Vaccination": ["Hib3", "HepB3", "RotaC", "RCV1"]
        }
        
        for area, sub_branches in health_areas.items():
            area_item = health_area_tree.insert("", "end", text=area)
            for sub_branch in sub_branches:
                health_area_tree.insert(area_item, "end", text=sub_branch)
        
        #query health areas
        query_button = Button(health_area_window, text="Query Data", command=query_health_area)
        query_button.pack(pady=5)

   #BY YEAR:
    elif selection == "By Year":
        year_window = Toplevel(root)
        year_window.title("Select Year")
        year_window.geometry("200x150")
        
        #year query
        query = "SELECT DISTINCT year FROM death_factors WHERE year > 1990"
        cursor.execute(query)
        years = [row[0] for row in cursor.fetchall()]
        
        #query the selected year
        def explore_year():
            selected_year = year_combobox.get()
            print("Exploring data for year:", selected_year)
            
            handle_year_query(selected_year)
        
        #year selection 
        year_var = StringVar()
        year_combobox = ttk.Combobox(year_window, textvariable=year_var)
        year_combobox['values'] = tuple(years)
        year_combobox.pack(pady=10)
        
        #Explore year
        explore_button = Button(year_window, text="Explore", command=explore_year)
        explore_button.pack(pady=5)

#BY REGION:
    elif selection == "By Region":
        region_window = Toplevel(root)
        region_window.title("Select Regions")
        region_window.geometry("300x200")

        query = "SELECT DISTINCT subregion FROM country_facts ORDER BY subregion"
        cursor.execute(query)
        regions = [row[0] for row in cursor.fetchall()]

        selected_region = StringVar()  
        selected_region.set(None)  

        radio_frame = Frame(region_window)
        radio_frame.pack()

        for region in regions:
            Radiobutton(radio_frame, text=region, variable=selected_region, value=region).pack(anchor=W)

        explore_button = Button(region_window, text="Explore", command=lambda: handle_region_query(selected_region.get()))
        explore_button.pack(pady=5)



#BY YEAR page query:
def handle_year_query(selected_year):
    #query
    query = f"SELECT country_facts.subregion, country_facts.country, child_health.prob_death_under5, death_factors.alcohol, life_expectancy.le_at_birth, maternal_health.maternal_deaths, vaccination.hib3 FROM country_facts JOIN child_health ON country_facts.code=child_health.code JOIN death_factors ON child_health.code=death_factors.code JOIN life_expectancy ON country_facts.code=life_expectancy.code JOIN maternal_health ON country_facts.code=maternal_health.code JOIN vaccination ON country_facts.code=vaccination.code WHERE child_health.year={selected_year} AND death_factors.year={selected_year} AND life_expectancy.year={selected_year} AND maternal_health.year={selected_year} AND vaccination.year={selected_year}"
    
    cursor.execute(query)
    data = cursor.fetchall()

    #result window
    result_window = Toplevel()
    result_window.title("Query Result")
    result_window.geometry("800x400")

    #TREE
    tree = ttk.Treeview(result_window, show="headings")
    tree.pack(fill="both", expand=True)  
    
    tree["columns"] = ("Region","Country", "Probability of Death Under 5", "Alcohol Deaths", "Life Expectancy at Birth", "Maternal Deaths", "Hib3 Vaccination")

    tree.heading("Region", text="Region", anchor="center")
    tree.heading("Country", text="Country", anchor="center")
    tree.heading("Probability of Death Under 5", text="Probability of Death Under 5", anchor="center")
    tree.heading("Alcohol Deaths", text="Alcohol Deaths", anchor="center")
    tree.heading("Life Expectancy at Birth", text="Life Expectancy at Birth", anchor="center")
    tree.heading("Maternal Deaths", text="Maternal Deaths", anchor="center")
    tree.heading("Hib3 Vaccination", text="Hib3 Vaccination", anchor="center")

    #insert in tree
    for row in data:
        tree.insert("", "end", values=row)

    #COUNTRY FILTER 
    def filter_by_country():
        selected_country = country_combobox.get()
        if selected_country:
            # Filter data based on the selected country
            filtered_data = [row for row in data if row[1] == selected_country]
            # Clear the treeview
            for item in tree.get_children():
                tree.delete(item)
            # Insert filtered data into treeview
            for row in filtered_data:
                tree.insert("", "end", values=row)

    #REGION FILTER
    def filter_by_region():
        selected_region = region_combobox.get()
        if selected_region:
            filtered_data = [row for row in data if row[0] == selected_region]
            for item in tree.get_children():
                tree.delete(item)
            for row in filtered_data:
                tree.insert("", "end", values=row)

    # Create a frame for filter controls
    filter_frame = Frame(result_window)
    filter_frame.pack(side=BOTTOM, pady=10)

    #Combobox-country
    country_var = StringVar()
    country_combobox = ttk.Combobox(filter_frame, textvariable=country_var)
    country_combobox['values'] = sorted(set(row[1] for row in data))
    country_combobox.pack(side=LEFT, padx=5)

    #country filter button
    filter_country_button = Button(filter_frame, text="Filter by Country", command=filter_by_country)
    filter_country_button.pack(side=LEFT, padx=5) 

    #query for subregion list
    cursor.execute("SELECT DISTINCT subregion FROM country_facts")
    subregions = [row[0] for row in cursor.fetchall()]

    #combobox-region
    region_var = StringVar()
    region_combobox = ttk.Combobox(filter_frame, textvariable=region_var)
    region_combobox['values'] = sorted(set(subregions))
    region_combobox.pack(side=LEFT, padx=5)

    #region filter button
    filter_region_button = Button(filter_frame, text="Filter by Region", command=filter_by_region)
    filter_region_button.pack(side=LEFT, padx=5)


#BY REGION query:
def handle_region_query(selected_region):
    #query
    query = """
    SELECT country_facts.subregion,
           child_health.year,
           round(AVG(child_health.prob_death_under5),2) AS "Avg prob. of death in children under 5",
           round(AVG(death_factors.alcohol),2) AS "Avg death by alcohol",
           round(AVG(life_expectancy.le_at_birth),2) AS "Avg life expectancy at birth"
    FROM country_facts
    JOIN child_health ON country_facts.code = child_health.code
    JOIN death_factors ON child_health.code = death_factors.code
    JOIN life_expectancy ON country_facts.code = life_expectancy.code
    WHERE country_facts.subregion = '{}'
    GROUP BY country_facts.subregion, child_health.year
    """.format(selected_region)

    print("Query:", query)  
    #execute uery
    cursor.execute(query)
    data = cursor.fetchall()

    #result window
    result_window = Toplevel()
    result_window.title("Query Result")
    result_window.geometry("800x400")

    #tree
    tree = ttk.Treeview(result_window)
    tree.pack(padx=10, pady=10)


    tree["columns"] = ("Subregion", "Year", "Avg prob. of death in children under 5", "Avg death by alcohol", "Avg life expectancy at birth")

    tree.heading("Subregion", text="Subregion", anchor="center")
    tree.heading("Year", text="Year", anchor="center")
    tree.heading("Avg prob. of death in children under 5", text="Avg prob. of death in children under 5", anchor="center")
    tree.heading("Avg death by alcohol", text="Avg death by alcohol", anchor="center")
    tree.heading("Avg life expectancy at birth", text="Avg life expectancy at birth", anchor="center")

    #insert into tree
    for row in data:
        tree.insert("", "end", values=row)

#end code and final packing 
root = Tk()
root.title("World Health Database")


main_label = Label(root, text="Explore Global Health Information:")
main_label.pack(pady=20)

dbvar = StringVar()
db_combobox = ttk.Combobox(root, textvariable=dbvar)
db_combobox['values'] = ('By Health Area', 'By Country', 'By Year', 'By Region') 
db_combobox.pack(padx=20, pady=10)

explore_button = ttk.Button(root, text="Go", command=explore_db)
explore_button.pack(padx=20,pady=10)

root.mainloop() 

Exploring data for year: 2000
